# GRA Agents: Loop-Free Demo / Агенты GRA: демонстрация без зацикливания

EN: This notebook shows a minimal `GraMultiAgentSystem` with two agents and a simple GRA-style reset.

RU: Этот ноутбук демонстрирует минимальную `GraMultiAgentSystem` с двумя агентами и простым GRA-сбросом.

In [ ]:
# EN:
# Basic imports.
# RU:
# Базовые импорты.

# !pip install -q gra-core

from gra_core import GraAgent, GraMultiAgentSystem


In [ ]:
# EN:
# Define a simple policy: it just echoes the state and counts steps.
# RU:
# Определяем простую политику: она эхо-возвращает состояние и считает шаги.

def counting_policy(state, memory):
    # EN: internal counter
    # RU: внутренний счётчик
    count = memory.get("count", 0) + 1
    new_memory = {"count": count}
    action = {"seen_state": state, "step": count}
    return action, new_memory


def reset_memory(memory):
    # EN: GRA-like cognitive reset (forget all internal history)
    # RU: GRA-сброс когнитивного состояния (забываем всю историю)
    return {}


In [ ]:
# EN:
# Create two agents with the same policy and reset function.
# RU:
# Создаём двух агентов с одинаковой политикой и функцией сброса.

agent_a = GraAgent("AgentA", counting_policy, reset_fn=reset_memory)
agent_b = GraAgent("AgentB", counting_policy, reset_fn=reset_memory)

agents = {"A": agent_a, "B": agent_b}


In [ ]:
# EN:
# Define an optional meta-reset for the whole multi-agent system.
# Here it just prints a message.
# RU:
# Определяем опциональный мета-сброс для всей мультиагентной системы.
# Здесь он просто печатает сообщение.

def meta_reset_fn(agent_dict):
    print("EN: Meta-reset applied to all agents")
    print("RU: Мета-сброс применён ко всем агентам")

system = GraMultiAgentSystem(agents=agents, meta_reset=meta_reset_fn)


## Run a few steps / Прогон нескольких шагов

EN: We will run several steps, inspect internal memory, then apply a global reset.

RU: Запустим несколько шагов, посмотрим на внутреннюю память, затем применим глобальный сброс.

In [ ]:
# EN:
# Run 3 steps with an evolving global state.
# RU:
# Запускаем 3 шага с меняющимся глобальным состоянием.

global_state = {"value": 0}

for t in range(3):
    global_state["value"] = t
    actions = system.step(global_state)
    print(f"EN: Step {t}, actions: {actions}")
    print(f"RU: Шаг {t}, действия: {actions}")
    print("EN: Agent memories:")
    print("RU: Память агентов:")
    for name, agent in system.agents.items():
        print(f"  {name}: {agent.memory}")
    print("-")


## Apply global reset / Применяем глобальный сброс

EN: Now apply `global_reset()` and check that internal memories are cleared.

RU: Теперь вызываем `global_reset()` и проверяем, что внутренняя память очищена.

In [ ]:
# EN:
# Apply GRA-style reset to all agents.
# RU:
# Применяем GRA-сброс ко всем агентам.

system.global_reset()

print("EN: Memories after global_reset():")
print("RU: Память после global_reset():")
for name, agent in system.agents.items():
    print(f"  {name}: {agent.memory}")


## One more step after reset / Ещё один шаг после сброса

EN: After reset, counters should start from 1 again.

RU: После сброса счётчики должны снова стартовать с 1.

In [ ]:
# EN:
# Step once more after reset.
# RU:
# Делаем ещё один шаг после сброса.

global_state["value"] = 99
actions = system.step(global_state)
print("EN: Actions after reset:")
print("RU: Действия после сброса:")
print(actions)

print("EN: Memories after one post-reset step:")
print("RU: Память после одного шага после сброса:")
for name, agent in system.agents.items():
    print(f"  {name}: {agent.memory}")
